In [2]:
from keras.models import load_model
model = load_model('Model.h5')

In [3]:
from pyzbar.pyzbar import decode
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras import layers
from time import perf_counter 
import os
import cv2
import csv
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.models import Sequential
# from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing import image
import datetime
from time import gmtime, strftime
# # import easyocr
# # import imutils
import pywhatkit

In [4]:
batch_size = 16
img_height = 360
img_width = 480

In [5]:
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'D:/MLModel/our_dataset_big/test',
    seed=40,
    image_size= (img_height, img_width),
    batch_size=batch_size)

class_names = testing_ds.class_names
print(class_names)
 
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

Found 664 files belonging to 2 classes.
['class1', 'class2']


In [6]:
#Original
# My camera fps is 30. 
import cv2
import os
import time

cam = cv2.VideoCapture(0)
# path = 'D:\Y\Accident-No'
# path2 = 'D:\Y\Accident-Yes'

last_recorded_time = time.time()# this keeps track of the last time a frame was processed
count=0
plt.figure(figsize=(15, 15))

# i=0 #increasing count after every loop
# j=0 # counting 3 second after accident occurs
k=[] #for storing value of barcode if accident occuers
# m=1 #if loop goes inside the accident then it doesnt go back insaide the accident before verification

while count <30:
#     i+=1
    #x=strftime("%d-%m-%Y %z")# ':' not allowed in file name
    curr_time = time.time() # grab the current time
    # keep these three statements outside of the if statement, so we 
    # can still display the camera/video feed in real time
    suc, frame = cam.read()
    frame = cv2.resize(frame,(200,200),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
    #plt.imshow(frame)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #operation like 'imshow' on image
    cv2.imshow('image', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
    if curr_time - last_recorded_time >= 1:# it has been at least 5 seconds
        count+=1
        Y = image.img_to_array(frame)
        X = tf.expand_dims(Y,axis=0)
        val = model.predict(X)
        print(val)        

#         img = images[i].numpy().astype("uint8")
#         img_array = tf.keras.utils.img_to_array(frame)
#         img_array = tf.expand_dims(img_array, 0) # Create a batch
#         predictions = model.predict(img_array)
#         score = tf.nn.softmax(predictions[0])
        
#         x = class_names[np.argmax(score)]
            
#         ax = plt.subplot(5, 5, i+1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title([i])
#         plt.axis("off")
        
        
        if val[0][1] > 0.5:
            print("Non-Acc")
        #if predictions[0][1] > 0.5:
#            cv2.imwrite(os.path.join('D:\Y\Accident-No' , F"{count}.jpg"), frame)
        if(val[0][1]<=0.5):
            for i in range (10):
                suce, frame = cam.read()
                #cv2.imshow('img',frame1)
                print(decode(frame))
                for barcode in decode(frame):
                    my_data=barcode.data.decode('utf-8')
                    pts=np.array([barcode.polygon],np.int32)
                    pts=pts.reshape((-1,1,2))
                    cv2.polylines(frame,[pts],True,(255,0,255),5)
                    pts2=barcode.rect
                    cv2.putText(frame,my_data,(pts2[0],pts2[1]),cv2.FONT_HERSHEY_PLAIN,0.9,(255,0,255),2)
                    if(my_data):
                        k.append(my_data)
                    cv2.waitKey(1000)
            
            for i in range (10):
                suce, frame = cam.read()
                #cv2.imshow('img',frame1)
                for barcode in decode(frame):
                    my_data=barcode.data.decode('utf-8')
                    pts=np.array([barcode.polygon],np.int32)
                    pts=pts.reshape((-1,1,2))
                    cv2.polylines(frame,[pts],True,(255,0,255),5)
                    pts2=barcode.rect
                    cv2.putText(frame,my_data,(pts2[0],pts2[1]),cv2.FONT_HERSHEY_PLAIN,0.9,(255,0,255),2)
                    if (my_data in k):

                    append_data=[
                         [my_data,'0']
                                ]
                    cv2.imwrite(os.path.join('D:\Y\Accident-Yes' , F"{count}.jpg" ), frame)
                    file=open('car_names.csv','a',newline='')
                    writer=csv.writer(file)
                    writer.writerows(append_data)
                    file.close()
                    x=strftime("%H %M")
                    x = [int(i) for i in x.split()]  
                    pywhatkit.sendwhatmsg('+919553652018', 'Accident happened!!!! \nConcerning ' + my_data + ' vehicle number at location ' + 'XXXX '+ ' \nPlease hurry', x[0],x[1]+2 )
                    break

        last_recorded_time = curr_time
        
cam.release()
cv2.destroyAllWindows()

1/1 [==============================] - 10s 10s/step
[[0.46111876 0.53888124]]
Non-Acc
1/1 [==============================] - 0s 69ms/step
[[0.4485128  0.55148715]]
Non-Acc
1/1 [==============================] - 0s 62ms/step
[[0.48484638 0.5151537 ]]
Non-Acc
1/1 [==============================] - 0s 45ms/step
[[0.5746831  0.42531687]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1/1 [==============================] - 0s 64ms/step
[[0.65773356 0.34226647]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1/1 [==============================] - 0s 60ms/step
[[0.6187941 0.3812059]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1/1 [==============================] - 0s 53ms/step
[[0.64049226 0.35950777]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1/1 [==============================] - 0s 54ms/step
[[0.5817791 0.4182209]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1/1 [==============================] - 0s 61ms/step
[[0.5741884  0.42581153]]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
1/1 [==============================] - 0s 65ms/step
[[0.4441999  0.55580014]]
Non-Acc
1/1 [=======

<Figure size 1500x1500 with 0 Axes>

In [1]:
!pip install digidevice

ERROR: Could not find a version that satisfies the requirement digidevice (from versions: none)
ERROR: No matching distribution found for digidevice


In [9]:
from geopy.geocoders import Nominatim
 
# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")
 
# entering the location name
getLoc = loc.geocode("IIT KANPUR")
 
# printing address
print(getLoc.address)
 
# printing latitude and longitude
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

IIT Kanpur, NH34, Kanpur, Kanpur Nagar, Uttar Pradesh, 208012, India
Latitude =  26.5091677 

Longitude =  80.248235
